In [20]:
import pandas as pd

In [21]:
mcc_parquet = 'data/mcc.parquet'
sales_parquet = 'data/sales.parquet'


In [22]:
df_mcc = pd.read_parquet(mcc_parquet, engine='pyarrow')

In [23]:
df_mcc.head()

,mcc,edited_description,combined_description,usda_description,irs_description,irs_reportable
0,742,Veterinary Services,Veterinary Services,Veterinary Services,Veterinary Services,Yes
1,763,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Cooperative,Yes
2,780,"Horticultural Services, Landscaping Services","Horticultural Services, Landscaping Services",Horticultural Services,Landscaping Services,Yes
3,1520,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors,Yes
4,1711,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,"Heating, Plumbing, A/C",Yes


In [24]:
# Visualização do dataset de vendas por document_id, estado e data/hora
df_sales = pd.read_parquet(sales_parquet, engine='pyarrow')



In [25]:
df_sales.head()

,document_id,date_time,value,card_number,type,mcc,state
0,9132021237731236867,2022-12-08 01:09:53.352600,240.0,,Pix,5499,RN
1,9132021237731236867,2022-12-04 03:04:08.115900,88.0,,Pix,5499,RN
2,9132021237731236867,2022-12-07 03:07:05.865600,76.0,,Pix,5499,RN
3,9132021237731236867,2022-12-05 02:45:01.539400,48.0,,Pix,5499,RN
4,9132021237731236867,2022-12-08 03:10:59.259900,8.0,,Pix,5499,RN


In [26]:
# Pegando só o document_id e mcc
document_id_mcc = df_sales[['document_id', 'mcc', 'state']]

In [27]:
# Carregamento do dataset de MCC / Descrição de estabelecimentos com base no MCC
mcc_description = df_mcc[['mcc', 'irs_description']]

In [28]:
# Visualização de todos os irs_descriptions
comercial_descriptions = mcc_description['irs_description'].unique()
comercial_descriptions

array(['Veterinary Services', 'Agricultural Cooperative',
       'Landscaping Services', 'General Contractors',
       'Heating, Plumbing, A/C', 'Electrical Contractors',
       'Masonry, Stonework, and Plaster', 'Carpentry Contractors',
       'Roofing/Siding, Sheet Metal', 'Concrete Work Contractors',
       'Special Trade Contractors',
       'Miscellaneous Publishing and Printing',
       'Typesetting, Plate Making, and Related Services',
       'Specialty Cleaning', 'Airlines', 'Car Rental',
       'Hotels/Motels/Inns/Resorts', None, 'Railroads',
       'Commuter Transport, Ferries', 'Passenger Railways',
       'Ambulance Services', 'Taxicabs/Limousines', 'Bus Lines',
       'Motor Freight Carriers and Trucking - Local and Long Distance, Moving and Storage Companies, and Local Delivery Services ',
       'Courier Services ',
       'Public Warehousing and Storage - Farm Products, Refrigerated Goods, Household Goods, and Storage ',
       'Cruise Lines', 'Boat Rentals and Leases',

In [29]:
supermakets_mcc = mcc_description[mcc_description['irs_description'] == 'Grocery Stores, Supermarkets']
supermakets_mcc.head()

,mcc,irs_description
760,5411,"Grocery Stores, Supermarkets"


In [30]:
df_sales = pd.read_parquet(sales_parquet, engine='pyarrow')


In [31]:
# Criação de dataset de vendas por tipos de estabelecimentos
sales_per_mcc_description = pd.merge(df_sales, supermakets_mcc, on='mcc', how='inner')

In [32]:
sales_per_mcc_description.head()

,document_id,date_time,value,card_number,type,mcc,state,irs_description
0,5695527918944804612,2023-03-17 16:47:24.959200,280.0,,Pix,5411,SC,"Grocery Stores, Supermarkets"
1,5695527918944804612,2023-01-30 18:16:19.936400,68.0,,Pix,5411,SC,"Grocery Stores, Supermarkets"
2,5695527918944804612,2023-02-11 18:22:41.237300,287.2,,Pix,5411,SC,"Grocery Stores, Supermarkets"
3,5695527918944804612,2023-04-01 21:23:36.940000,440.0,,Pix,5411,SC,"Grocery Stores, Supermarkets"
4,5695527918944804612,2022-12-11 19:22:35.025400,132.0,,Pix,5411,SC,"Grocery Stores, Supermarkets"


In [33]:
all_document_ids = sales_per_mcc_description['document_id'].unique()

In [34]:
all_document_ids

array([5695527918944804612, 9202060645083883366, 7523807556076637323,
       4574000117844462501, 1690472938955678638, 5564683868409713852])

### Criação de dataset com dados mockados

In [35]:
import numpy as np
from datetime import datetime


In [36]:
items = ['Maçã', 'Banana', 'Leite', 'Pão', 'Arroz', 'Feijão', 'Café', 'Sabonete']


In [37]:
# Função para calcular o preço total
def calculate_price(row):
    return item_prices[row['item']] * row['qtd']

In [38]:
df_mocked = pd.DataFrame({'document_id': np.random.choice(all_document_ids, size=10000, replace=True)})
df_mocked['irs_description'] = 'Grocery Stores, Supermarkets'
df_mocked['item'] = np.random.choice(items, size=len(df_mocked))
df_mocked['qtd'] = np.random.randint(1, 12, size=len(df_mocked))

In [39]:
item_prices = {'Maçã': 1.20, 'Banana': 0.80, 'Leite': 4.28, 'Pão': 0.60, 'Arroz': 21.0, 'Feijão': 12.0, 'Café': 15.70, 'Sabonete': 3.20}
# Adicionar a coluna de preço
df_mocked['price'] = df_mocked.apply(calculate_price, axis=1)

In [40]:
# Gerando datas aleatórias entre 2022 e 2023
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 12, 31)
time_delta = end_date - start_date
df_mocked['data'] = start_date + time_delta * np.random.rand(len(df_mocked))

df_mocked['dia'] = df_mocked['data'].dt.day
df_mocked['mes'] = df_mocked['data'].dt.month
df_mocked['ano'] = df_mocked['data'].dt.year
df_mocked['hora'] = df_mocked['data'].dt.hour
df_mocked['document_id'] = df_mocked['document_id'].astype(str)



In [41]:
df_mocked.head()

,document_id,irs_description,item,qtd,price,data,dia,mes,ano,hora
0,1690472938955678638,"Grocery Stores, Supermarkets",Maçã,7,8.4,2022-11-28 11:51:16.776456,28,11,2022,11
1,9202060645083883366,"Grocery Stores, Supermarkets",Arroz,4,84.0,2022-02-25 02:25:32.384556,25,2,2022,2
2,9202060645083883366,"Grocery Stores, Supermarkets",Leite,5,21.4,2023-09-10 18:30:27.489020,10,9,2023,18
3,9202060645083883366,"Grocery Stores, Supermarkets",Pão,5,3.0,2022-06-02 06:52:19.409245,2,6,2022,6
4,7523807556076637323,"Grocery Stores, Supermarkets",Pão,3,1.8,2022-11-17 07:00:53.225760,17,11,2022,7


In [42]:
df_mocked.to_parquet('mocked_sale_items.parquet', index=False)
